# Introduction to ESG Bulk - Python

ESG stands for Environmental, Social and (Corporate) Governance data.

Refinitiv Data Platform (RDP) provides simple web based API access to a broad range of content, including ESG content and ESG content in bulk.

With growing popularity of socially conscious investing, Refinitiv offers one of the most comprehensive Environmental, Social and Governance (ESG) databases in the industry, covering over 80% of global market cap, across more than 450 different ESG metrics, with history going back to 2002. Customers looking to download our ESG content can do so through our bulk API service in Refinitiv Data Platform (RDP). RDP is a cloud based API that provides a single access point to all Refinitiv content.

ESG data is the first content made available in our bulk API service known as Client File Store (CFS). This capability allows our customers to download our entire history of ESG coverage. To more about how the ESG Bulk Service works in Refinitiv Data Platform, please visit:

https://developers.refinitiv.com/refinitiv-data-platform/refinitiv-data-platform-apis/docs

Within RDP family of service, ESG Bulk is part of Client File Store (CFS) - based section of service, find out more at:

https://developers.refinitiv.com/en/api-catalog/refinitiv-data-platform/refinitiv-data-platform-apis

Let us now focus on the programmatic interaction with ESG Bulk RDP service.

### Valid Credentials - Hard-coded or Read From File

For this example, we can either read one's valid credentials from a file (that can be shared by many code examples) or replace the commented out hard-coded credentials with one's valid assigned credentials and comment the read from file step

In [1]:
import requests, json, time, getopt, sys

### Either hard code valid credentials (uncomment)
#USERNAME = "VALIDUSER"
#PASSWORD = "VALIDPASSWORD"
#CLIENT_ID = "SELFGENERATEDCLIENTID"

### or 

### read credentials from file
credFile = open("..\creds\credFileHuman.txt","r")    # one per line
                                                #--- RDP MACHINE ID---
                                                #--- LONG PASSWORD---
                                                #--- GENERATED CLIENT ID---

USERNAME = credFile.readline().rstrip('\n')
PASSWORD = credFile.readline().rstrip('\n')
CLIENT_ID = credFile.readline().rstrip('\n')

credFile.close()

# Make sure that creds are read in correctly
#print("USERNAME="+str(USERNAME))
#print("PASSWORD="+str(PASSWORD))
#print("CLIENT_ID="+str(CLIENT_ID))

### Set Application Constants

In [ ]:
# Set Application Constants
RDP_AUTH_VERSION = "/v1"
RDP_ESG_BULK_VERSION = "/v1"
RDP_BASE_URL = "https://api.refinitiv.com"
RDP_ESG_BUCKET = "ESG"
CATEGORY_URL = "/auth/oauth2"
ENDPOINT_URL = "/token"
CLIENT_SECRET = ""
TOKEN_FILE = "token.txt"
SCOPE = "trapi"

### Define Token Handling and Obtain a Valid Token

Having a valid token is a pre-requisite to requesting of any RDP content, and will be passed into the next steps.

In [2]:
TOKEN_ENDPOINT = RDP_BASE_URL + CATEGORY_URL + RDP_AUTH_VERSION + ENDPOINT_URL

def _requestNewToken(refreshToken):
    if refreshToken is None:
        tData = {
            "username": USERNAME,
            "password": PASSWORD,
            "grant_type": "password",
            "scope": SCOPE,
            "takeExclusiveSignOnControl": "true"
        };
    else:
        tData = {
            "refresh_token": refreshToken,
            "grant_type": "refresh_token",
        };

    # Make a REST call to get latest access token
    response = requests.post(
        TOKEN_ENDPOINT,
        headers = {
            "Accept": "application/json"
        },
        data = tData,
        auth = (
            CLIENT_ID,
            CLIENT_SECRET
        )
    )
    
    if response.status_code != 200:
        raise Exception("Failed to get access token {0} - {1}".format(response.status_code, response.text));

    # Return the new token
    return json.loads(response.text);

def saveToken(tknObject):
    tf = open(TOKEN_FILE, "w+");
    print("Saving the new token");
    # Append the expiry time to token
    tknObject["expiry_tm"] = time.time() + int(tknObject["expires_in"]) - 10;
    # Store it in the file
    json.dump(tknObject, tf, indent=4)
    
def getToken():
    try:
        print("Reading the token from: " + TOKEN_FILE);
        # Read the token from a file
        tf = open(TOKEN_FILE, "r+")
        tknObject = json.load(tf);

        # Is access token valid
        if tknObject["expiry_tm"] > time.time():
            # return access token
            return tknObject["access_token"];

        print("Token expired, refreshing a new one...");
        tf.close();
        # Get a new token from refresh token
        tknObject = _requestNewToken(tknObject["refresh_token"]);

    except Exception as exp:
        print("Caught exception: " + str(exp))
        print("Getting a new token using Password Grant...");
        tknObject = _requestNewToken(None);

    # Persist this token for future queries
    saveToken(tknObject)
    print("Token is: " + tknObject["access_token"])
    # Return access token
    return tknObject["access_token"];

accessToken = getToken();
print("Have token now");

Reading the token from: token.txt
Token expired, refreshing a new one...
Caught exception: Failed to get access token 400 - {"error":"invalid_grant"   } 
Getting a new token using Password Grant...
Saving the new token
Token is: eyJ0eXAiOiJhdCtqd3QiLCJhbGciOiJSUzI1NiIsImtpZCI6ImRMdFd2Q0tCSC1NclVyWm9YMXFod2pZQ2t1eDV0V2ZSS2o4ME9vcjdUY28ifQ.eyJkYXRhIjoie1wiY2lwaGVydGV4dFwiOlwiYjJNeElaUEJMbEJEVEJMV0d5aXFQYkZYN1RJM2hEVUNtQk1VOFhUQXRFcTYweVJMWjFyd2lCcTgwd0JsX2I5enlpbmM0blRIS05zaGMwOGx2ZFJObjQxUTlZODYxUElCakhURUVjT08yQ3k0OE5PbWtOX2x4MGtMQ2VaNG4tcU5GT1F1RXozSEJCRC1QcDlsaFJoYmNWSkRGUEkwNkN2UVp4bkVTMVdIZmwwLW1mU21vb2VWSnFXYzE4aTFCaVlsUkhaTEJET0hOOWpSajVDYkV4S1BKOHVYWVJWbWplbVhURDFhUHNUUnZtek5MaFB6RXZOSG5nWkJTMm00QkdmamFyOUpzM1YzTGZCVG5sTWpmX3VOSFpRTFJaOG93ekxXdVg0RzBlYkpuTjZ6cHdkQ01XcXJfam1YR1FRT0NoVTd1c0hBS2c3cUhXVXByYjhudEVyQUVSeFdWb2I2RmVQODlhVmVQdE9tSEdNbXpnYkY3WG1wNVZPVjdJRFQzZGt0OGVUem02Rm1vSHFhMkFqSTdkYkh5cThWQnNCaHdqR0pzRmduSVFXZ3FNVGRjTThXTjZ2SXZ2RzhwZ1R3eW9IUWxiYlR4VWtheVRxUVhaSWw1ME5L

## Request Available File Sets

The purpose of ESG bulk service is obtaining ESG content in bulk.  The content is available as:

•	A full JSON data file containing history for all measures and all organizations.
•	A delta JSON data file that contains only incremental changes to the universe since last week. 

A customer can examine the available File Sets and is expected to:

•	Build the initial ESG representation with the full files
•	Apply delta, changes, as they become available
•	Fill a gap if the retrival was not completed and the content missed remains available

This step serves to verify the type of the file, for example:
•	ESGRawFullScheme
•	ESGScoresFull
•	ESGScoresWealthFull

In [3]:
FILESET_ENDPOINT = RDP_BASE_URL+'/file-store'+RDP_ESG_BULK_VERSION + '/file-sets?bucket='+ RDP_ESG_BUCKET
FILESET_ID = ''
WHICH_FILESET_ID_TO_STORE = 1   # 10 are returned per page

def requestFileSets(token, withNext, skipToken, attributes):   
    global FILESET_ENDPOINT
    global FILESET_ID
    global WHICH_FILESET_ID_TO_STORE   
    
    print("Obtaining FileSets in ESG Bucket...")
  
    FILESET_ENDPOINT = RDP_BASE_URL+'/file-store'+RDP_ESG_BULK_VERSION + '/file-sets?bucket='+ RDP_ESG_BUCKET
    
    querystring = {}
    payload = ""
    jsonfull = ""
    jsonpartial = ""
    
    headers = {
            'Content-Type': "application/json",
            'Authorization': "Bearer " + token,
            'cache-control': "no-cache"
    }

    if attributes:
        FILESET_ENDPOINT = FILESET_ENDPOINT + attributes
    if withNext:
        FILESET_ENDPOINT = FILESET_ENDPOINT + '&skipToken=' +skipToken
        
    response = requests.request("GET", FILESET_ENDPOINT, data=payload, headers=headers, params=querystring)
    
    if response.status_code != 200:
        if response.status_code == 401:   # error when token expired
                accessToken = getToken();     # token refresh on token expired
                headers['Authorization'] = "Bearer " + accessToken
                response = requests.request("GET", FILESET_ENDPOINT, data=payload, headers=headers, params=querystring)
         
    print('Raw response=');
    print(response);
    
    if response.status_code == 200:
        jsonFullResp = json.loads(response.text)
        print('Parsed json response=');
        print(json.dumps(jsonFullResp, indent=2));
        # We are going to store FileSet ID of the second File Set retrieved for future reference
        FILESET_ID = jsonFullResp['value'][WHICH_FILESET_ID_TO_STORE]['id']
        print('FILESET_ID stored is: '+str(FILESET_ID))
        return jsonFullResp; 
    else:
        return '';

jsonFullResp = requestFileSets(accessToken, False, '','');

Obtaining FileSets in ESG Bucket...
Raw response=
<Response [200]>
Parsed json response=
{
  "value": [
    {
      "id": "4013-1c69-e0e65b84-a977-00094191f69c",
      "name": "RFT-ESG-Raw-Full-SchemeB-Env-Delta-2021-01-03",
      "bucketName": "ESG",
      "packageId": "4b6c-def9-dd1c991c-8535-6f9a61df9fc8",
      "attributes": [
        {
          "name": "ResultCount",
          "value": "54259"
        },
        {
          "name": "ContentType",
          "value": "ESG Raw Full B"
        }
      ],
      "files": [
        "4336-8d29-409a03eb-813f-e6867748347d",
        "456d-d2bf-3ce9f56f-872b-59393d61fcae"
      ],
      "numFiles": 2,
      "availableFrom": "2021-01-03T13:12:36Z",
      "availableTo": "2021-01-17T13:12:36Z",
      "status": "READY",
      "created": "2021-01-03T13:12:36Z",
      "modified": "2021-01-03T13:13:02Z"
    },
    {
      "id": "4022-f0c0-e3968404-9146-1b496d4e11b4",
      "name": "RFT-ESG-Sources-Current-Delta-2021-01-10",
      "bucketName": "ESG

### Paginate Through the Available FileSets
(interrupt at any point)

In [4]:
i = 1
while "@nextLink" in jsonFullResp: 
    print('<<< Iteraction: '+str(i)+' >>>  More exists: '+ jsonFullResp['@nextLink'] + ', skipToken is: ' + jsonFullResp['@nextLink'][-62:]+'\n')
    jsonFullResp = requestFileSets(accessToken, True, jsonFullResp['@nextLink'][-62:],'');
    i+=1;
print('Last response without next=');
print(jsonFullResp);

<<< Iteraction: 1 >>>  More exists: /file-store/v1/file-sets?bucket=ESG&skipToken=ZmlsZXNldElkPTQ0YjUtYTJiNy0wZDExNDAxMC1hM2Y1LTEzMmIzMWZkNDViZA, skipToken is: ZmlsZXNldElkPTQ0YjUtYTJiNy0wZDExNDAxMC1hM2Y1LTEzMmIzMWZkNDViZA

Obtaining FileSets in ESG Bucket...
Raw response=
<Response [200]>
Parsed json response=
{
  "value": [
    {
      "id": "44e0-490a-2a5471e1-8ce9-dfecd4666d0d",
      "name": "RFT-ESG-Sources-Full-Delta-2021-01-03",
      "bucketName": "ESG",
      "packageId": "4308-bc80-2054dc20-83db-6224911311d0",
      "attributes": [
        {
          "name": "ContentType",
          "value": "ESG Sources"
        },
        {
          "name": "ResultCount",
          "value": "5337"
        }
      ],
      "files": [
        "4caa-2c19-988fba80-bc20-275ee390efd4"
      ],
      "numFiles": 1,
      "availableFrom": "2021-01-03T14:12:51Z",
      "availableTo": "2021-01-17T14:12:50Z",
      "status": "READY",
      "created": "2021-01-03T14:12:51Z",
      "modified": "2021-

### Retrieve FileSets of Specific File Type (Filter By Attribute)
The file types may change over time, at the time of this writing, the available FileSets are of types:

* ESG Raw Full A
* ESG Raw Full B
* ESG Raw Current A
* ESG Raw Current B
* ESG Sources
* ESG Raw Wealth Standard

* Symbology Cusip
* Symbology SEDOL
* Symbology Organization
* Symbology Instrument Quote


In [14]:
requestFileSets(accessToken, False, '','&attributes=ContentType:Symbology Cusip');

Obtaining FileSets in ESG Bucket...
Reading the token from: token.txt
Raw response=
<Response [200]>
Parsed json response=
{
  "value": [
    {
      "id": "41d2-f1a3-df5d16dd-b672-38a1acd5f676",
      "name": "RFT-ESG-Symbology-Cusip-Init-2020-12-31",
      "bucketName": "ESG",
      "packageId": "466c-c470-22a4cab3-8d0c-770cdfa5fc45",
      "attributes": [
        {
          "name": "ContentType",
          "value": "Symbology Cusip"
        }
      ],
      "files": [
        "4d80-f8a4-432f6272-9323-75f75c08ca45"
      ],
      "numFiles": 1,
      "availableFrom": "2020-12-31T10:29:40Z",
      "availableTo": "2021-01-14T10:29:39Z",
      "status": "READY",
      "created": "2020-12-31T10:29:40Z",
      "modified": "2020-12-31T10:29:52Z"
    },
    {
      "id": "458d-60a7-cfe476de-9d35-117f124921aa",
      "name": "RFT-ESG-Symbology-Cusip-Init-2020-12-24",
      "bucketName": "ESG",
      "packageId": "466c-c470-22a4cab3-8d0c-770cdfa5fc45",
      "attributes": [
        {
       

### Retrieve Complete File Details of FileSet ID

In a previous step we have stored a FileSet ID that we are about to use for the demonstartion of this feature.

In [5]:
FILES_ENDPOINT = RDP_BASE_URL+'/file-store'+RDP_ESG_BULK_VERSION + '/files?filesetId='+ FILESET_ID
WHICH_FILE_ID_TO_STORE = 0
FILE_ID = ''
FILE_NAME = ''
 
def requestFileDetails(token):   
    global FILES_ENDPOINT
    global FILE_ID
    global FILE_NAME
    print("Obtaining File details..." )
  
    querystring = {}
    payload = ""
    jsonfull = ""
    jsonpartial = ""
    
    headers = {
            'Content-Type': "application/json",
            'Authorization': "Bearer " + token,
            'cache-control': "no-cache"
    }
        
    response = requests.request("GET", FILES_ENDPOINT, data=payload, headers=headers, params=querystring)
    
    if response.status_code != 200:
        if response.status_code == 401:   # error when token expired
                accessToken = getToken();     # token refresh on token expired
                headers['Authorization'] = "Bearer " + accessToken
                response = requests.request("GET", FILES_ENDPOINT, data=payload, headers=headers, params=querystring)
         
    print('Raw response=');
    print(response);
    
    if response.status_code == 200:
        jsonFullResp = json.loads(response.text)
        print('Parsed json response=');
        print(json.dumps(jsonFullResp, indent=2));
        FILE_ID = jsonFullResp['value'][WHICH_FILE_ID_TO_STORE]['id']
        print('FILE_ID stored is: '+str(FILE_ID))
        FILE_NAME = jsonFullResp['value'][WHICH_FILE_ID_TO_STORE]['filename']
        print('FILE_NAME stored is: '+str(FILE_NAME))
        return jsonFullResp; 
    else:
        return '';

jsonFullResp = requestFileDetails(accessToken);

Obtaining File details...
Raw response=
<Response [200]>
Parsed json response=
{
  "value": [
    {
      "id": "4400-9f13-7f9f2f37-8e8d-f8d26840be99",
      "filename": "RFT-ESG-Scores-Wealth-Full-Init-2021-01-03.jsonl.gz",
      "filesetId": "4fd4-316c-badb4383-80e4-ab784d63eac5",
      "storageLocation": {
        "url": "https://a206464-prod-esg.s3.amazonaws.com/ESG_Scores/2021/01/03/RFT-ESG-Scores-Wealth-Full-Init-2021-01-03.jsonl.gz",
        "@type": "s3"
      },
      "created": "2021-01-03T13:57:30Z",
      "modified": "2021-01-03T13:57:30Z",
      "href": "https://api.refinitiv.com/file-store/v1/files/4400-9f13-7f9f2f37-8e8d-f8d26840be99/stream",
      "fileSizeInBytes": 16104609
    }
  ]
}
FILE_ID stored is: 4400-9f13-7f9f2f37-8e8d-f8d26840be99
FILE_NAME stored is: RFT-ESG-Scores-Wealth-Full-Init-2021-01-03.jsonl.gz


###  Stream File via File Id using Redirect

In [6]:
import shutil

FILES_STREAM_ENDPOINT = RDP_BASE_URL+'/file-store'+RDP_ESG_BULK_VERSION + '/files/'+ FILE_ID+ '/stream'
 
def requestFileDownload(token):   
    global FILES_STREAM_ENDPOINT
    print("Obtaining File ... " + FILES_STREAM_ENDPOINT)
  
    filename = FILE_NAME
    chunk_size = 1000
    
    headers = {
            'Authorization': 'Bearer ' + token,
            'cache-control': "no-cache",
            'Accept': '*/*'
    }
        
    response = requests.request("GET", FILES_STREAM_ENDPOINT, headers=headers, stream=True, allow_redirects=True)
    
    if response.status_code != 200:
        if response.status_code == 401:   # error when token expired
                accessToken = getToken();     # token refresh on token expired
                headers['Authorization'] = "Bearer " + accessToken
                response = requests.request("GET",FILES_STREAM_ENDPOINT, headers=headers, stream=True, allow_redirects=True)

         
    print('Response code=' + str(response.status_code));
    
    if response.status_code == 200:
        print('Processing...')
        with open(filename, 'wb') as fd:
            shutil.copyfileobj(response.raw, fd) 
        print('Look for gzipped file named: '+ filename + ' in current directory')
        response.connection.close()
        
    return; 


requestFileDownload(accessToken);

Obtaining File ... https://api.refinitiv.com/file-store/v1/files/4400-9f13-7f9f2f37-8e8d-f8d26840be99/stream
Reading the token from: token.txt
Token expired, refreshing a new one...
Saving the new token
Token is: eyJ0eXAiOiJhdCtqd3QiLCJhbGciOiJSUzI1NiIsImtpZCI6ImRMdFd2Q0tCSC1NclVyWm9YMXFod2pZQ2t1eDV0V2ZSS2o4ME9vcjdUY28ifQ.eyJkYXRhIjoie1wiY2lwaGVydGV4dFwiOlwib09hc2J0aWY0OXZkVC1jZDVsZVJXYUc4dzZWaTYxVjN4TXNBWWlOWV9rd3pCdGplMWpKbEZrUW9VQkg3Zmx0blZQeVMwTFU2RVN6ZVU5UVplLU1YbFFaSm1kYlNVTmRXcmQ4d0MzNnFIdFpxSFVDVkxwbGpkaGZ6UzE2UE1yYTFVNk1FNzVEZVp6Um5iLXNCQWxsdllyZktBY3N4WmNWQjh1ZExjWXFXVVRPMXNaSndwalk5b0NHNnNxT1dXdWVqdWk1ZzZacVgweTc4TG05cjV1cmkyODl5SDhnRjBzVVhvSV94M2tmS2RZMnd0Z25OdjRSOExacl9oMTQzUS1TOGNyMGx2X0hQRWhmS0w3Ukc2RklPcmZWNzEtdkRkNkdpdXBWd3FwRkUySDhKMGJfQjYwd01VMHBfOVN6Y0lXN2UwQ2dJZC01c3lNcXlXZ1djVHByX1d0aU9uUk9BajhiQlYxQmFqUldveFY2Y3U3VHB1QUM1aGh4aTdrVnJ2OWZmSGRybUxhQVVwaTdqX0hhVWt3aGhrbXBhbElpcjNmcnZmU1BlQ3VMMW9WbVhhSVpGaVRqMlk5dm1uOUxSNmNqM2pGYVVidWZOZE5WcXpJTjVobXVkN3hqTmtpQWU1aGU3

### Get File Location (Step 1 of 2)

In [17]:
import shutil

FILES_STREAM_ENDPOINT = RDP_BASE_URL+'/file-store'+RDP_ESG_BULK_VERSION + '/files/'+ FILE_ID+ '/stream?doNotRedirect=true'
DIRECT_URL = ''
 
def requestFileLocation(token):   
    
    global FILES_STREAM_ENDPOINT
    global DIRECT_URL
    
    print("Obtaining File ... " + FILES_STREAM_ENDPOINT)
  
    filename = FILE_NAME
    chunk_size = 1000
    
    headers = {
            'Authorization': 'Bearer ' + token,
            'cache-control': "no-cache",
            'Accept': '*/*'
    }
        
    response = requests.request("GET", FILES_STREAM_ENDPOINT, headers=headers, stream=False, allow_redirects=False)
    
    if response.status_code != 200:
        if response.status_code == 401:   # error when token expired
                accessToken = getToken();     # token refresh on token expired
                headers['Authorization'] = "Bearer " + accessToken
                response = requests.request("GET",FILES_STREAM_ENDPOINT, headers=headers, stream=False, allow_redirects=False)

         
    print('Response code=' + str(response.status_code));
    
    if response.status_code == 200:
        jsonFullResp = json.loads(response.text)
        print('Parsed json response=');
        print(json.dumps(jsonFullResp, indent=2));
        DIRECT_URL = jsonFullResp['url'];
        print('File Direct URL is: '  +str(DIRECT_URL)+ '|||');
        
    return; 


requestFileLocation(accessToken);

Obtaining File ... https://api.refinitiv.com/file-store/v1/files/47cb-2400-e6f524d4-8703-c3941bf8ab58/stream?doNotRedirect=true
Reading the token from: token.txt
Response code=200
Parsed json response=
{
  "url": "https://a206464-prod-esg.s3.amazonaws.com/ESG_Symbology/2020/12/24/RFT-ESG-Symbology-Cusip-Init-2020-12-24.jsonl.gz?x-request-Id=536d09e8-091e-4186-b7c7-b7a3e06eba2a&x-package-id=466c-c470-22a4cab3-8d0c-770cdfa5fc45&x-client-app-id=GE-A-01103867-3-603&x-file-name=RFT-ESG-Symbology-Cusip-Init-2020-12-24.jsonl.gz&x-fileset-id=458d-60a7-cfe476de-9d35-117f124921aa&x-bucket-name=ESG&x-uuid=GENTC-25929&x-file-Id=47cb-2400-e6f524d4-8703-c3941bf8ab58&x-fileset-name=RFT-ESG-Symbology-Cusip-Init-2020-12-24&x-event-external-name=cfs-claimCheck-download&X-Amz-Security-Token=IQoJb3JpZ2luX2VjEGsaCXVzLWVhc3QtMSJIMEYCIQD%2Ftq0Gm8hMY8rR0Ag4aFsCbP2NGl5Po5rupc5VofvGdAIhAMrnxq8I48iN0QP8mNAOdXj1LNflv2yrWJeCMU1AZHBqKskDCDQQAhoMNjQyMTU3MTgxMzI2IgxC%2B7h4k42jyuQfpFAqpgNcYR9DULH7s5EOnrAJY69xSemjO3toR

### Download File From File Location (Step 2 of 2)

In [20]:
from urllib.parse import urlparse, parse_qs
def requestDirectFileDownload(token):   
    
    global DIRECT_URL
    print("Obtaining File from URL... " + DIRECT_URL)
    
    #Parse out URL parameters for submission into requests
    url_obj = urlparse(DIRECT_URL)
    parsed_params = parse_qs(url_obj.query)
    # extract the URL without query parameters
    parsed_url = url_obj._replace(query=None).geturl()

    response = requests.get(parsed_url, params=parsed_params,stream=True)
        
    if response.status_code != 200:
        if response.status_code == 401:   # error when token expired
                accessToken = getToken();     # token refresh on token expired
                headers['Authorization'] = "Bearer " + accessToken
                response = requests.get(parsed_url, params=query)

         
    print('Response code=' + str(response.status_code));        
  
    filename = 'another_'+FILE_NAME    
    
    if response.status_code == 200:
        print('Processing...')
        with open(filename, 'wb') as fd:
            shutil.copyfileobj(response.raw, fd) 

        print('Look for gzipped file named: '+ filename + ' in current directory')
        response.connection.close()
        
    return; 


requestDirectFileDownload(accessToken);

Obtaining File from URL... https://a206464-prod-esg.s3.amazonaws.com/ESG_Symbology/2020/12/24/RFT-ESG-Symbology-Cusip-Init-2020-12-24.jsonl.gz?x-request-Id=536d09e8-091e-4186-b7c7-b7a3e06eba2a&x-package-id=466c-c470-22a4cab3-8d0c-770cdfa5fc45&x-client-app-id=GE-A-01103867-3-603&x-file-name=RFT-ESG-Symbology-Cusip-Init-2020-12-24.jsonl.gz&x-fileset-id=458d-60a7-cfe476de-9d35-117f124921aa&x-bucket-name=ESG&x-uuid=GENTC-25929&x-file-Id=47cb-2400-e6f524d4-8703-c3941bf8ab58&x-fileset-name=RFT-ESG-Symbology-Cusip-Init-2020-12-24&x-event-external-name=cfs-claimCheck-download&X-Amz-Security-Token=IQoJb3JpZ2luX2VjEGsaCXVzLWVhc3QtMSJIMEYCIQD%2Ftq0Gm8hMY8rR0Ag4aFsCbP2NGl5Po5rupc5VofvGdAIhAMrnxq8I48iN0QP8mNAOdXj1LNflv2yrWJeCMU1AZHBqKskDCDQQAhoMNjQyMTU3MTgxMzI2IgxC%2B7h4k42jyuQfpFAqpgNcYR9DULH7s5EOnrAJY69xSemjO3toReCmEPStxmr41vVCDV8%2Foacwygzh3LQunFgInznvLQQumzPXRupZXRbBuaIwLQBE29Il5XRHITBSnAd8mfBOE0jOrM3HaUXFwwGe%2Bb5Jz3g24ej4uFtgffYjhLLZek0rZjqc%2BZmBPK7jJ7bnoni%2BD9H6FVnAvc8WTRLEo5HmdLtfGK8IFLr%